In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
df = pd.read_csv('Advertising.csv')

In [12]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [13]:
X = df.drop('sales', axis=1)

In [14]:
y = df['sales']

In [15]:
from sklearn.preprocessing import PolynomialFeatures

In [16]:
polynomial_converter = PolynomialFeatures(degree=3, include_bias=False)

In [17]:
poly_features = polynomial_converter.fit_transform(X)

In [18]:
poly_features.shape

(200, 19)

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=101)

In [21]:
X_train.shape

(140, 19)

In [22]:
from sklearn.preprocessing import StandardScaler

In [23]:
scaler = StandardScaler()

In [24]:
scaler.fit(X_train)

# fit only on training set to avoid data leakage from test set
# it means that model will not have seen any information from the test set

,copy,True
,with_mean,True
,with_std,True


In [25]:
X_train_std = scaler.transform(X_train)

In [26]:
X_test_std = scaler.transform(X_test)

In [27]:
X_train_std[0]

array([ 0.49300171, -0.33994238,  1.61586707,  0.28407363, -0.02568776,
        1.49677566, -0.59023161,  0.41659155,  1.6137853 ,  0.08057172,
       -0.05392229,  1.01524393, -0.36986163,  0.52457967,  1.48737034,
       -0.66096022, -0.16360242,  0.54694754,  1.37075536])

In [28]:
poly_features[0]

array([2.30100000e+02, 3.78000000e+01, 6.92000000e+01, 5.29460100e+04,
       8.69778000e+03, 1.59229200e+04, 1.42884000e+03, 2.61576000e+03,
       4.78864000e+03, 1.21828769e+07, 2.00135918e+06, 3.66386389e+06,
       3.28776084e+05, 6.01886376e+05, 1.10186606e+06, 5.40101520e+04,
       9.88757280e+04, 1.81010592e+05, 3.31373888e+05])

In [29]:
from sklearn.linear_model import Ridge


In [30]:
help(Ridge)

Help on class Ridge in module sklearn.linear_model._ridge:

class Ridge(sklearn.base.MultiOutputMixin, sklearn.base.RegressorMixin, _BaseRidge)
 |  Ridge(
 |      alpha=1.0,
 |      *,
 |      fit_intercept=True,
 |      copy_X=True,
 |      max_iter=None,
 |      tol=0.0001,
 |      solver='auto',
 |      positive=False,
 |      random_state=None
 |  )
 |
 |  Linear least squares with l2 regularization.
 |
 |  Minimizes the objective function::
 |
 |  ||y - Xw||^2_2 + alpha * ||w||^2_2
 |
 |  This model solves a regression model where the loss function is
 |  the linear least squares function and regularization is given by
 |  the l2-norm. Also known as Ridge Regression or Tikhonov regularization.
 |  This estimator has built-in support for multi-variate regression
 |  (i.e., when y is a 2d-array of shape (n_samples, n_targets)).
 |
 |  Read more in the :ref:`User Guide <ridge_regression>`.
 |
 |  Parameters
 |  ----------
 |  alpha : {float, ndarray of shape (n_targets,)}, default=1.

In [31]:
ridge_model = Ridge(alpha=10)

In [32]:
ridge_model.fit(X_train_std, y_train)

,alpha,10
,fit_intercept,True
,copy_X,True
,max_iter,None
,tol,0.0001
,solver,'auto'
,positive,False
,random_state,None


In [34]:
test_predictions = ridge_model.predict(X_test_std)

In [40]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [41]:
MAE = mean_absolute_error(y_test, test_predictions)

In [43]:
MAE

0.5774404204714166

In [44]:
RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))

In [45]:
RMSE

np.float64(0.8946386461319648)

In [46]:
from sklearn.linear_model import RidgeCV

In [70]:
ridge_cv_model = RidgeCV(alphas=(0.1, 1.0, 10.0), scoring='neg_mean_absolute_error')
# it will choose the best alpha value from the given list of alphas
# it will use neg_mean_absolute_error as scoring metric

In [71]:
ridge_cv_model.fit(X_train_std, y_train)

,alphas,"(0.1, ...)"
,fit_intercept,True
,scoring,'neg_mean_absolute_error'
,cv,None
,gcv_mode,None
,store_cv_results,False
,alpha_per_target,False


In [72]:
ridge_cv_model.alpha_

np.float64(0.1)

In [61]:
from sklearn.metrics import get_scorer_names

In [73]:
#get_scorer_names()

In [74]:
test_predictions = ridge_cv_model.predict(X_test_std)
MAE = mean_absolute_error(y_test, test_predictions)
RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))

In [75]:
MAE

0.42737748843375084

In [76]:
RMSE

np.float64(0.6180719926926787)

In [ ]:
#alpha=0.1 is the best value from the given list of alphas

#compared to alpha=10, the MAE and RMSE values are lower now
# so the model is performing better now
# MAE was 0.58, now it is 0.43
# RMSE was 0.89, now it is 0.61 

In [77]:
ridge_cv_model.coef_

array([ 5.40769392,  0.5885865 ,  0.40390395, -6.18263924,  4.59607939,
       -1.18789654, -1.15200458,  0.57837796, -0.1261586 ,  2.5569777 ,
       -1.38900471,  0.86059434,  0.72219553, -0.26129256,  0.17870787,
        0.44353612, -0.21362436, -0.04622473, -0.06441449])

In [78]:
ridge_cv_model.best_score_

# it is the best score achieved during cross-validation

-0.3749223340292961

In [83]:
from sklearn.linear_model import LassoCV
# Lasso is another regularization technique like Ridge

In [100]:
#lasso_cv_model = LassoCV(eps=0.001, n_alphas=100, cv=5, max_iter=1000000)
lasso_cv_model = LassoCV(eps=0.1, alphas=100, cv=5)

# eps = 0.001 is the length of the path. alpha_min / alpha_max = 0.001 
# alpha_max is the maximum alpha value in the path and alpha_min is the minimum

# n_alphas is the number of alphas to try
# cv is the number of cross-validation folds

# max_iter is the maximum number of iterations to run
# sometimes the model may not converge in the default number of iterations

In [101]:
lasso_cv_model.fit(X_train_std, y_train)

,eps,0.1
,n_alphas,'deprecated'
,alphas,100
,fit_intercept,True
,precompute,'auto'
,max_iter,1000
,tol,0.0001
,copy_X,True
,cv,5
,verbose,False
,n_jobs,None


In [88]:
X_train.shape

(140, 19)

In [89]:
X_train_std.shape

(140, 19)

In [102]:
lasso_cv_model.alpha_

np.float64(0.4943070909225831)

In [103]:
test_predictions = lasso_cv_model.predict(X_test_std)

In [105]:
MAE = mean_absolute_error(y_test, test_predictions)
MAE

0.6541723161252868

In [106]:
RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))

In [107]:
RMSE

np.float64(1.1308001022762548)

In [108]:
lasso_cv_model.coef_

array([1.002651  , 0.        , 0.        , 0.        , 3.79745279,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])

In [ ]:
#here we can see that some of the coefficients are exactly zero
# this means that those features are not important for the model

#even though on ridge regression we got better results
# but here we can see which features are important and which are not
# so we can use this for feature selection

# we can drop the features which have zero coefficients


#lasso model is more interpretable than ridge model
#it can be useful for feature selection
# but if the main goal is to get the best performance
# then ridge regression is better in this case
# because it gives lower MAE and RMSE values
# but if we want to know which features are important
# then lasso regression is better because it gives sparse coefficients

In [118]:
lasso_cv_model = LassoCV(eps=0.001, alphas=100, cv=5, max_iter=1000000)
lasso_cv_model.fit(X_train_std, y_train)

,eps,0.001
,n_alphas,'deprecated'
,alphas,100
,fit_intercept,True
,precompute,'auto'
,max_iter,1000000
,tol,0.0001
,copy_X,True
,cv,5
,verbose,False
,n_jobs,None


In [122]:
test_predictions = lasso_cv_model.predict(X_test_std)

In [124]:
lasso_cv_model.alpha_

np.float64(0.004943070909225831)

In [120]:
MAE = mean_absolute_error(y_test, test_predictions)

In [121]:
MAE

0.4335034618590074

In [125]:
RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))

In [126]:
RMSE

np.float64(0.6063140748984036)

In [128]:
lasso_cv_model.coef_
# here we can see that more coefficients are non-zero now
# so the model is using more features now
# performance is better than before


array([ 4.86023329,  0.12544598,  0.20746872, -4.99250395,  4.38026519,
       -0.22977201, -0.        ,  0.07267717, -0.        ,  1.77780246,
       -0.69614918, -0.        ,  0.12044132, -0.        , -0.        ,
       -0.        ,  0.        ,  0.        , -0.        ])

In [129]:
from sklearn.linear_model import ElasticNetCV

In [130]:
# ElasticNet is a combination of Ridge and Lasso regression

In [135]:
elastic_cv_model = ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1],
                                eps=0.001, 
                                alphas=100,
                                max_iter=1000000)

#l1_ratio is the ratio of Lasso to Ridge
# l1_ratio=1 means only Lasso
# l1_ratio=0 means only Ridge

# we can give a list of l1_ratio values to try
# it will choose the best one based on cross-validation

#alphas is the list of alpha values to try

In [136]:
elastic_cv_model.fit(X_train_std, y_train)

,l1_ratio,"[0.1, 0.5, ...]"
,eps,0.001
,n_alphas,'deprecated'
,alphas,100
,fit_intercept,True
,precompute,'auto'
,max_iter,1000000
,tol,0.0001
,cv,None
,copy_X,True
,verbose,0


In [144]:
elastic_cv_model.alpha_
# best alpha value 
# same as lasso_cv_model.alpha_

np.float64(0.004943070909225831)

In [145]:
elastic_cv_model.l1_ratio
# returnt the list of l1_ratio values tried

[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]

In [146]:
elastic_cv_model.l1_ratio_
# best l1_ratio value from the given list

# here we can see that l1_ratio=1 is the best value
# so it means that Lasso is the best model for this dataset

np.float64(1.0)

In [147]:
#attribute with underscore at the end means it is the best value!

In [149]:
test_predictions = elastic_cv_model.predict(X_test_std)

In [150]:
MAE = mean_absolute_error(y_test, test_predictions)

In [151]:
MAE

0.4335034618590074

In [ ]:
# In conclusion, instead of using Ridge or Lasso alone
# we can use ElasticNet which combines both Ridge and Lasso regression models 
# and it will choose the best model based on cross-validation